# Creating DCGAN from Sctrach

## importing Libraries

In [ ]:

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

## Model

### Creating Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, channel_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(in_channels=channel_img,
                               out_channels=features_d,
                               kernel_size=4,
                               stride=2,
                               padding=1
                               ), # 64x64
            nn.LeakyReLU(0.2),
            self._block(in_chanels=features_d,
                        out_channels=features_d*2,
                        kernel_size=4,
                        stride=2,
                        padding=1
                        ),#16x16
            self._block(in_chanels=features_d*2,
                        out_channels=features_d*4,
                        kernel_size=4,
                        stride=2,
                        padding=1
                        ),#8x8
            self._block(in_chanels=features_d*4,
                        out_channels=features_d*8,
                        kernel_size=4,
                        stride=2,
                        padding=1
                        ),#4x4
            nn.Conv2d(
                in_channels=features_d*8,
                out_channels=1,
                kernel_size=4,
                stride=2,
                padding=0
            ),#1x1
            nn.Sigmoid()
        )

    def _block(self,
               in_chanels,
               out_channels,
               kernel_size,
               stride,
               padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels=in_chanels,
                out_channels=out_channels,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )
    def forward(self,x):
        return self.disc(x)

### Creating Generator

In [ ]:
   
class Generator(nn.Module):
    def __init__(self, z_din,channel_img,features_g):
        super(Generator,self).__init__()
        self.net = nn.Sequential(
            self._block(
                in_chanels=z_din,
                out_channels=features_g*16,
                kernel_size=4,
                stride=1,
                padding=0
            ),#N x f_g*16 x 4 x 4
            self._block(
                in_chanels=features_g*16,
                out_channels=features_g*8,
                kernel_size=4,
                stride=2,
                padding=1
            ),#8x8
            self._block(
                in_chanels=features_g*8,
                out_channels=features_g*4,
                kernel_size=4,
                stride=2,
                padding=1
            ),#16x16
            self._block(
                in_chanels=features_g*4,
                out_channels=features_g*2,
                kernel_size=4,
                stride=2,
                padding=1
            ),#32x32
            nn.ConvTranspose2d(
                in_channels=features_g*2,
                out_channels=channel_img,
                kernel_size=4,
                stride=2,
                padding=1,
            ),
            nn.Tanh(),
        )
        
    def _block(self,
               in_chanels,
               out_channels,
               kernel_size,
               stride,
               padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels=in_chanels,
                out_channels=out_channels,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(0.2),
        )
    def forward(self,x):
        return self.net(x)

### Creatring InItial Weights

In [ ]:
        
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m,
                      (
                          nn.Conv2d,
                          nn.ConvTranspose2d,
                          nn.BatchNorm2d
                        )):
            nn.init.normal_(m.weight.data,0.0,0.02)

### Testing the model

In [ ]:
def test():
    N,in_channels,H,W = 8,3,64,64
    z_dim = 100
    x = torch.randn((N,in_channels,H,W))
    
    disc = Discriminator(in_channels,8)
    initialize_weights(disc)
    assert disc(x).shape == (N,1,1,1)
    
    gen = Generator(z_dim, in_channels, 8)
    initialize_weights(gen)
    z = torch.randn((N, z_dim, 1, 1))
    assert gen(z).shape == (N, in_channels, H, W)
    
    print("Success")
    
test()

## Hapreparameter

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available else 'cpu')
LEARNING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNEL_IMG = 1
Z_DIM = 100
NUM_EPOCHS=5
FEATURES_DISC, FEATHURE_DEN = 64

In [ ]:
transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            [
                0.5 for _ in range(CHANNEL_IMG)
            ],
            [
                0.5 for _ in range(CHANNEL_IMG)
            ]
        ),
    ]
)

## Dataset

In [ ]:
datasets = datasets.MNIST(
    'dataset/',
    train=True,
    transform=transforms,
    download=True
)